In [1]:
import os
import requests
import json 
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas import json_normalize
import time
from defillama import DefiLlama
from datetime import datetime, timedelta
llama = DefiLlama()
import datetime as dt
import pytz
import sqlite3

In [2]:
def get_net_flows(chain_name:str):
    """
    Takes in the name of a block chain, checks 
    all bridges to see if the chain is used on that bridge
    aggregates inflow/outflow of money to chain via each bridge
    """
    ids = []
    aggregated_data = {}

    bridges = "https://bridges.llama.fi/bridges?includeChains=true"
    bridges = requests.get(bridges).json()
    
    for i in range(len(bridges['bridges'])):
        if f"{chain_name}" in bridges['bridges'][i]['chains']:
            ids.append(bridges['bridges'][i]['id'])
    
    for id in ids:
        url = f"https://bridges.llama.fi/bridgevolume/{chain_name}?id={id}"
        vol_data = requests.get(url).json()[-60:]

        for entry in vol_data:
            date = datetime.utcfromtimestamp(int(entry['date']))
            eastern = pytz.timezone("US/Eastern")
            date = date.replace(tzinfo=pytz.utc).astimezone(eastern)
            date = date.strftime('%Y-%m-%d')
            net_inflow = entry['depositUSD'] - entry['withdrawUSD']

            if date in aggregated_data:
                aggregated_data[date] += net_inflow
            else:
                aggregated_data[date] = net_inflow
    sorted_flows = sorted(aggregated_data.items(), key=lambda x: x[0])

    return sorted_flows

In [4]:
def get_stables(chain_name:str):
    """
    Stable ids is a list 1-160 for all stablecoin ids on defillama
    chain name + stablecoin id is passed into the request and the 
    aggregate stable coin value on each chain is calculated 
    """
    stable_ids = list(range(1,160))
    aggregated_stables = {}
       
    for id in stable_ids:
        url = f"https://stablecoins.llama.fi/stablecoincharts/{chain_name}?stablecoin={id}"
        stables_data = requests.get(url).json()[-60:]  # Get the last 60 records

        if stables_data:
            for entry in stables_data:
                date = datetime.utcfromtimestamp(int(entry['date']))
                eastern = pytz.timezone("US/Eastern")
                date = date.replace(tzinfo=pytz.utc).astimezone(eastern).strftime('%Y-%m-%d')

                # Correctly using the safe access method here
                total_circulating_peggedUSD = entry.get('totalCirculating', {}).get('peggedUSD', 0)

                # Aggregate using the safely accessed value
                if date in aggregated_stables:
                    aggregated_stables[date] += total_circulating_peggedUSD
                else:
                    aggregated_stables[date] = total_circulating_peggedUSD
    sorted_agg_stables = sorted(aggregated_stables.items(), key = lambda x:x[0])

    return sorted_agg_stables

In [5]:
def get_tvl(chain_name:str):
    """
    Take in chain name, pass to TVL request, gather TVL across chains
    """
    tvl = {}
    tvl_url = F"https://api.llama.fi/v2/historicalChainTvl/{chain_name}"
    tvl_data = requests.get(tvl_url).json()[-60:]

    for entry in tvl_data:
        date = datetime.utcfromtimestamp(int(entry['date']))
        eastern = pytz.timezone("US/Eastern")
        date = date.replace(tzinfo=pytz.utc).astimezone(eastern)
        date = date.strftime('%Y-%m-%d')

        
        tvl[date] = entry['tvl']
    
    sorted_tvl = sorted(tvl.items(), key = lambda x:x[0])

    return sorted_tvl

In [6]:
def get_dex_vol(chain_name:str):
    """
    Get aggregate dex volume done on each chain aggregated across all dexes
    """
    dex_volume = {'24H': 0, '7D': 0, '30D': 0}
    dex_url = f"https://api.llama.fi/overview/dexs/{chain_name}?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"

    try:
        dex_data = requests.get(dex_url).json()
        for entry in dex_data['protocols']:
            dex_volume['24H'] += round(entry.get('total24h', 0),2) if entry.get('total24h') is  not None else 0 
            dex_volume['7D'] += round(entry.get('total7d', 0),2) if entry.get('total7d') is  not None else 0 
            dex_volume['30D'] += round(entry.get('total30d', 0),3) if entry.get('total30d') is  not None else 0 
    except Exception as e:
        print(f"An error occurred: {e}")

    return dex_volume

### Create Initial SQL Database to hold the past 60 days of data for each function:

In [7]:
def create_tables():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS net_flows
                      (chain_name TEXT, date TEXT, net_inflow REAL, PRIMARY KEY (chain_name, date))''')
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS Stablecoins
                      (chain_name TEXT, date TEXT, total_stables REAL, PRIMARY KEY (chain_name, date))''')
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS TVL
                      (chain_name TEXT, date TEXT, TVL REAL, PRIMARY KEY (chain_name, date))''')
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS Dex_Volume
                      (chain_name TEXT, volume_24h REAL, volume_7d REAL, volume_30d REAL, PRIMARY KEY (chain_name))''')
    
    
    conn.commit()
    conn.close()

In [8]:
def insert_data(table_name, data_dict):
    """
    Insert data into the specified table.
    
    Parameters:
    - table_name: The name of the table to insert data into.
    - data_dict: A dictionary where keys are column names and values are the data to insert.
    """
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Prepare column names and placeholders for values
    columns = ', '.join(data_dict.keys())
    placeholders = ', '.join(['?' for _ in data_dict])
    
    query = f'''INSERT OR REPLACE INTO {table_name} ({columns}) VALUES ({placeholders})'''
    cursor.execute(query, list(data_dict.values()))
    
    conn.commit()
    conn.close()

In [9]:
def fetch_and_insert_flows(chain_name):
    flows_data = get_net_flows(chain_name)  # Make sure this function returns the correct data format
    for date, net_inflow in flows_data:
        insert_data('net_flows', {  # Use the correct table name as defined in create_tables
            'chain_name': chain_name,
            'date': date,
            'net_inflow': net_inflow
        })


def fetch_and_insert_stablecoins(chain_name):
    stables_data = get_stables(chain_name)  # Assuming this returns a list of tuples (date, total_stables)
    for date, total_stables in stables_data:
        insert_data('Stablecoins', {
            'chain_name': chain_name,
            'date': date,
            'total_stables': total_stables
        })

# Example for fetching and inserting TVL and DEX Volume (simplified)
def fetch_and_insert_tvl(chain_name):
    tvl_data = get_tvl(chain_name)  # Assuming it returns sorted tuples (date, TVL)
    for date, tvl in tvl_data:
        insert_data('TVL', {
            'chain_name': chain_name,
            'date': date,
            'TVL': tvl
        })

def fetch_and_insert_dex_volume(chain_name):
    dex_volume_data = get_dex_vol(chain_name)  
    formatted_dex_volume_data = {
        'chain_name': chain_name,
        'volume_24h': dex_volume_data.get('24H', 0),  # Assume get_dex_vol returns dict with '24H', '7D', '30D'
        'volume_7d': dex_volume_data.get('7D', 0),
        'volume_30d': dex_volume_data.get('30D', 0),
    }
    insert_data('Dex_Volume', formatted_dex_volume_data)


In [10]:
# Iterate over all chains and fetch/insert data for each
chains = ['Ethereum', 'Solana', 'Base', 'Sei', 'Sui', 'Injective', 'Avalanche', 'Optimism']

create_tables()

for chain in chains:
    fetch_and_insert_flows(chain)
    fetch_and_insert_stablecoins(chain)
    fetch_and_insert_tvl(chain)
    fetch_and_insert_dex_volume(chain)

### Query Each Database to Inspect Data:

In [4]:
# ETH, SOL, Base, Avalanche, Optimism
def query_database_flows():
    conn = sqlite3.connect('C:/Users/Justin/projects/chain_data.db')
    cursor = conn.cursor()
    
    # Example: Query the net_flows table
    cursor.execute("SELECT * FROM net_flows")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

query_database_flows()

('Ethereum', '2023-08-20', 34.0)
('Ethereum', '2023-08-21', 0.0)
('Ethereum', '2023-08-22', 6741.0)
('Ethereum', '2023-08-23', 0.0)
('Ethereum', '2023-08-24', 0.0)
('Ethereum', '2023-08-25', 0.0)
('Ethereum', '2023-08-26', 0.0)
('Ethereum', '2023-08-27', 8039.0)
('Ethereum', '2023-08-28', 106.0)
('Ethereum', '2023-08-29', 100.0)
('Ethereum', '2023-08-30', 50.0)
('Ethereum', '2023-08-31', 34.0)
('Ethereum', '2023-09-01', 0.0)
('Ethereum', '2023-09-02', 0.0)
('Ethereum', '2023-09-03', 2357.0)
('Ethereum', '2023-09-04', 616765.0)
('Ethereum', '2023-09-05', 362160.0)
('Ethereum', '2023-09-06', 1113224.0)
('Ethereum', '2023-09-07', 125599.0)
('Ethereum', '2023-09-08', 102061.0)
('Ethereum', '2023-09-09', 112823.0)
('Ethereum', '2023-09-10', -71059.0)
('Ethereum', '2023-09-11', -132610.0)
('Ethereum', '2023-09-12', 300812.0)
('Ethereum', '2023-09-13', -279567.0)
('Ethereum', '2023-09-14', 508757.0)
('Ethereum', '2023-09-15', -180776.0)
('Ethereum', '2023-09-16', 296074.0)
('Ethereum', '2023-

In [11]:
# ETH, SOL, Base, Avalanche, Optimism, SUI
def query_database_stables():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Example: Query the net_flows table
    cursor.execute("SELECT * FROM Stablecoins")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

query_database_stables()

('Ethereum', '2023-12-24', 100595083.99)
('Ethereum', '2023-12-25', 67738494129.219986)
('Ethereum', '2023-12-26', 67672636951.189995)
('Ethereum', '2023-12-27', 67915582462.729996)
('Ethereum', '2023-12-28', 67937060189.37999)
('Ethereum', '2023-12-29', 67928559321.389984)
('Ethereum', '2023-12-30', 67904623094.16999)
('Ethereum', '2023-12-31', 67728359605.96999)
('Ethereum', '2024-01-01', 67862428530.43999)
('Ethereum', '2024-01-02', 68309424340.94002)
('Ethereum', '2024-01-03', 68531725971.29001)
('Ethereum', '2024-01-04', 68850964393.35002)
('Ethereum', '2024-01-05', 69291479990.37997)
('Ethereum', '2024-01-06', 69393582760.38998)
('Ethereum', '2024-01-07', 69556150920.29002)
('Ethereum', '2024-01-08', 69714655088.26003)
('Ethereum', '2024-01-09', 69734418809.99998)
('Ethereum', '2024-01-10', 69825504184.93997)
('Ethereum', '2024-01-11', 69970160463.81999)
('Ethereum', '2024-01-12', 70010228187.72997)
('Ethereum', '2024-01-13', 70104529167.7)
('Ethereum', '2024-01-14', 70057307503.

In [4]:
# ETH, SOL, Base, SEI, Sui, INJ, Avalanche, Optimism, 
def query_database_tvl():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Example: Query the net_flows table
    cursor.execute("SELECT * FROM TVL")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

query_database_tvl()

('Ethereum', '2023-12-28', 30610489342.97283)
('Ethereum', '2023-12-29', 29751828881.071396)
('Ethereum', '2023-12-30', 30039788050.67762)
('Ethereum', '2023-12-31', 30107479147.252426)
('Ethereum', '2024-01-01', 31387922717.0552)
('Ethereum', '2024-01-02', 31139542136.891754)
('Ethereum', '2024-01-03', 29445658278.377575)
('Ethereum', '2024-01-04', 30306379703.53779)
('Ethereum', '2024-01-05', 30099527782.84863)
('Ethereum', '2024-01-06', 30137272840.94493)
('Ethereum', '2024-01-07', 29948398089.118916)
('Ethereum', '2024-01-08', 30598405979.523666)
('Ethereum', '2024-01-09', 32250960914.627888)
('Ethereum', '2024-01-10', 34734965861.052986)
('Ethereum', '2024-01-11', 34379275281.37082)
('Ethereum', '2024-01-12', 33196505803.997425)
('Ethereum', '2024-01-13', 33408743304.29816)
('Ethereum', '2024-01-14', 33236232957.838055)
('Ethereum', '2024-01-15', 32896828794.9997)
('Ethereum', '2024-01-16', 32885170137.091927)
('Ethereum', '2024-01-17', 33254853033.46033)
('Ethereum', '2024-01-18'

In [13]:
# ETH, SOL, Base, Avalanche, Optimism, SUI
def query_database_dex_volume():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Example: Query the net_flows table
    cursor.execute("SELECT * FROM Dex_volume")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

query_database_dex_volume()

('Ethereum', 1791835945.0, 10419490125.0, 37214565802.602005)
('Solana', 620382843.0, 4596911177.0, 21339380484.003)
('Base', 43775831.0, 272545392.0, 1006562463.576)
('Sei', 792775.0, 9282833.0, 36591421.505)
('Sui', 49389097.0, 364726604.0, 2453839901.2289996)
('Injective', 21470201.0, 164674573.0, 637830745.849)
('Avalanche', 71899265.0, 584079116.0, 2506400562.2860003)
('Optimism', 73479896.0, 542129454.53, 1802855082.54)
